## Chapter 8: transformación de datos, unir, combinar y reformar
En muchas aplicaciones, los datos pueden extenderse a través de una serie de archivos o bases de datos, o organizarse
en un formulario que no es conveniente analizar.

In [1]:
import pandas as pd
import numpy as np
import sys

## Indexación jerárquica
La indexación jerárquica es una carácteristica importante de Pandas que le permite tener múltiples niveles de índice
en un eje. Otra forma de pensar al respecto es que proporciona una forma de trabajar con datos dimensionales
superiores en una forma dimensional inferior.

In [2]:
data = pd.Series(np.random.uniform(size=9),
                index=[list("aaabbccdd"),
                [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.292471
   2    0.254687
   3    0.713271
b  1    0.394561
   3    0.535408
c  1    0.867234
   2    0.369601
d  2    0.625910
   3    0.658556
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data["b"]

1    0.394561
3    0.535408
dtype: float64

In [5]:
data["b":"c"]

b  1    0.394561
   3    0.535408
c  1    0.867234
   2    0.369601
dtype: float64

In [6]:
data.loc[["b", "d"]]

b  1    0.394561
   3    0.535408
d  2    0.625910
   3    0.658556
dtype: float64

La selección es incluso posible desde un nivel "inner". Seleccionar todos los valores que tiene el valor `2` desde
el segundo nivel de índice:

In [7]:
data.loc[:, 2]

a    0.254687
c    0.369601
d    0.625910
dtype: float64

La indexación jerárquica juega un papel importante en la remodelación de datos y operaciones basadas en grupos,
como formar una tabla pivote. Por ejemplo, puede reorganizar estos datos en un DAtaFrame utilizando el método
`unstack`:

In [8]:
data.unstack()

,1,2,3
a,0.292471,0.254687,0.713271
b,0.394561,NaN,0.535408
c,0.867234,0.369601,NaN
d,NaN,0.625910,0.658556


La operación inversa es `stack`:

In [9]:
inv = data.unstack()
inv.stack()

a  1    0.292471
   2    0.254687
   3    0.713271
b  1    0.394561
   3    0.535408
c  1    0.867234
   2    0.369601
d  2    0.625910
   3    0.658556
dtype: float64

Con un DataFrame, cualquier eje puede tener un índice jerárquico:

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[list("aabb"), [1, 2, 1, 2]],
                    columns=[["Ohio", "Ohio", "Colorado"],
                            ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Los niveles jerárquicos pueden tener nombres (como cadenas o cualquier objeto Python). Si es así, esto aparecerán
en la salida de la consola:

In [11]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Esto nombres reemplazan al atributo `name`, que se usa solo con índices de un solo nivel.

Usar `nlevels` para ver cuántos niveles tiene un índice:

In [12]:
frame.index.nlevels

2

Con la indexación parcial de columnas, puede seleccionar de manera similar grupos de columnas:

In [13]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

El DataFrame anterior se puede recrear con `MultiIndex`:

In [14]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green","Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Niveles de reordenación y clasificación
A veces es posible que deba reorganizar el orden de los niveles en un eje u ordenar los datos por los valores en
un nivel específico. El método `swaplevel` toma dos números de nivel o nombres y devuelve un nuevo objeto con los
niveles intercambiados (pero los datos no están alterados):

In [15]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` de forma predeterminada, ordena los datos lexicográficamente utilizando todos los niveles de índice,
pero puede elegir usar solo un nivel único o un subconjunto de niveles para ordenar usando el argumento `level`:

In [16]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [17]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Resumen estadísticas por nivel
Muchas estadísticas descriptivas y resumidas sobre DataFrame y Series tienen una opción `level` en la que pueden
especificar el nivel que desean agragar en un eje en particular. Se puede agregar por nivel en filas o columnas.

In [18]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [19]:
frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexación con columnas de DataFrame
No es inusual querer usar una o más columnas de un DataFrame como índice de fila; alternativamente, es posible
que desee mover el índice de fila a las columnas del DataFrame.

In [20]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                     "c": ["one", "one", "one", "two", "two", "two", "two"],
                     "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


La función `set_index` creará un nuevo DataFrame utilizando una o más de sus columnas como índice:

In [21]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

De froma predeterminada, las colummas se elimina del DataFrame, aunque puede dejarlas con `drop=False`:

In [22]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index` hace lo contrario; los niveles de índice jerárquico se mueven a las columnas:

In [23]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### Combiar y fusionar conjuntos de datos
Los contenidos en los objetos de Pandas se pueden combinar de varias maneras:

`pandas.merge`: conecta filas en DataFrame en función de una o más claves. Esto sería familiar para los usuarios de
SQL u otras bases de datos relacionales, ya que implementa la operación de base de datos "_join_".

`pandas.concat`: concatenar o apilar objetos juntos a lo largo de un eje.

`combine_first`: dividir los datos superpuestos para completar los valores faltantes en un objeto con calores de otro.

Las operaciones `merge` o `join` combinan conjuntos de datos al vincular filas usando una o más claves. Estas
operaciones son particularmente importantes en bases de datos relaciones. La función `pandas.merge` es el principal
punto de entrada para usar estos algoritmos en sus datos.

In [24]:
df1 = pd.DataFrame({"key": list("bbacaab"),
                   "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": list("abd"),
                   "data2": pd.Series(range(3), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [26]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Si no se especifica la columna a unir, `pandas.merge` utiliza los nombres de columna superpuestas como las claves.
Sin embargo, es buena práctica especificar explícitamente:

In [27]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Si lo nombres de la columna son diferentes en cada objeto, puede especificarlos por separado:

In [28]:
df3 = pd.DataFrame({"lkey": list("bbacaab"),
                   "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": list("abd"),
                   "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Puede notar que a `"c"` y `"d"` le faltan valores y datos asociados en el resultado. Por defecto, `pd.merge` hacer una
unión `"inner"`; las claves en los resultados son las intersecciones, o el conjunto común que se encuentra en ambas
tablas. Otra opciones posibles son `"left"`, `"right"` y `"outer"`. La unión externa toma la unión de las claves,
combinando el efecto de aplicar ambas uniones izquierda y derecha:

In [29]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [30]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


Las fusiones _muchos a muchos_ forman el producto cartesiano de las claves.

In [31]:
df1 = pd.DataFrame({"key": list("bbacab"),
                   "data1": pd.Series(range(6), dtype="Int64")})

df2 = pd.DataFrame({"key": list("abadb"),
                   "data2": pd.Series(range(5), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [32]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,d,3
4,b,4


In [33]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,4
2,b,1,1
3,b,1,4
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [34]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,4
2,b,1,1
3,b,1,4
4,b,5,1
5,b,5,4
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [35]:
# pasar una lista para fusionar varias líneas
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                    "key2": ["one", "two", "one"],
                    "lval": pd.Series(range(3), dtype="Int64")})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                     "key2": ["one", "one", "one", "two"],
                     "rval": pd.Series(range(4, 8, 1), dtype="Int64")})

pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,0,4
1,foo,one,0,5
2,foo,two,1,<NA>
3,bar,one,2,6
4,bar,two,<NA>,7


In [36]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,0,one,4
1,foo,one,0,one,5
2,foo,two,1,one,4
3,foo,two,1,one,5
4,bar,one,2,one,6
5,bar,one,2,two,7


`pd.merge` tiene una opción `suffixes` para especificar cadenas para agregar nombres superpuestos en los objetos
DataFrame izquiero y derecho:

In [37]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,0,one,4
1,foo,one,0,one,5
2,foo,two,1,one,4
3,foo,two,1,one,5
4,bar,one,2,one,6
5,bar,one,2,two,7


### Fusión en el índice
En algunos casos, la claves de fusión en un DataFrame se encontrará en su índice. En ese caso, puede pasar
`left_index=True` o `right_index=True` (o ambos) para indicar que el índice debe usarse como la clave de fusión:

In [38]:
left1 = pd.DataFrame({"key": list("abaabc"),
                    "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=list("ab"))

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [39]:
right1

,group_val
a,3.5
b,7.0


In [40]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [41]:
# formar la unión de ellas con una unión externa
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Con datos inexados jerárquicamente, las cosas son más complicadas, ya que unir al índice es equivalente a una fusión
de múltiples claves:

In [42]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
                    "key2": [2000, 2001, 2002, 2001, 2002],
                    "data": pd.Series(range(5), dtype="Int64")})

right_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

righth = pd.DataFrame({"event1": pd.Series(range(0, 11, 2), dtype="Int64",
                                          index=right_index),
                     "event2": pd.Series(range(1, 12, 2), dtype="Int64",
                                         index=right_index)})

lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [43]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

En este caso, debe indicar varias columnas para fusionarse como una lista (tener en cuenta el manejo de valores de
índice duplicados con `"outer"`):

In [44]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [45]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
        right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


También es posible usar los índices de ambos lados de la fusión:

In [46]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index=list("ace"),
                    columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=list("bcde"),
                     columns=["Missouri", "Alabama"]).astype("Int64")

left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [47]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [48]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


DataFrame tiene un método de instancia `join` para simplificar la fusión por índice. También se puede usar para
combinar muchos objetos DataFrame que tienen los mismos o similares índices, pero columnas no superpuestas.

In [49]:
# simplificando el ejemplo anterior
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


Comparado con `pd.merge`, el método `join` realiza una unión izquierda en claves de unión de forma predeterminada.
También admite unir el índice de DataFrame pasando en una de las columnas la llamada a las columnas del DataFrame:

In [50]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Para fusiones simples de índice en índice, se puede pasar una lista de DataFrame a `join` como alternativa a usar
`pd.concat`:

In [51]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                      index=list("acef"),
                      columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [52]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [53]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


### Concatenando a lo largo de un eje
Otro tipo de operación de combinación de datos se denomina como _concatenación_ o _apilamiento_. La función
`concatenate` puede hacer esto con las matrices NumPy:

In [54]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [55]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

En el contecto de objetos pandas como Series y Dataframe, tener ejes etiquetados le permite generalizar aún más
las concatenación de la matriz. En particular, tiene una serie de preocupaciones adicionales:

- Si los objetos están indexados de manera diferentes en los otros ejes, ¿deberíamos combinar los distintos elementos en estos ejes o usar solo los valores en común?
- ¿Los fragmentos de datos concatenados deben ser identificados como tales en el objeto resultante?
- ¿El "eje de concatenación" contiene datos que deben conservarse? En muchos casos, las etiquetas enteras predeterminadas en un DataFrame se descartan mejor durante la contatenación.

La función `concat` en pandas proporciona una forma consistente en abordar cada una de estas preguntas. Supongamos
que tenemos tres series sin superposición de índice:

In [56]:
s1 = pd.Series([0, 1], index=list("ab"), dtype="Int64")
s2 = pd.Series([2, 3, 4], index=list("cde"), dtype="Int64")
s3 = pd.Series([5, 6], index=list("fg"), dtype="Int64")

In [57]:
# pega los valores por índice
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

Por defecto, `pd.concat` trabaja a lo largo `axis="index"` produciendo otra serie. Para columnas `axis="columns"`,
el resultado será un DataFrame:

In [58]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


En este casi no hay superposición en el otro eje, que como se puede ver en la unión (`"outer"`) de los índices.
En su lugar, puede cruzarlos pasando `join="inner"`:

In [59]:
s4 = pd.concat([s1, s2])
s4

a    0
b    1
c    2
d    3
e    4
dtype: Int64

In [60]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
c,<NA>,2
d,<NA>,3
e,<NA>,4


In [61]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


Un problema potencial es que las piezas concatenadas no son identificables en el resultado. Supongamos que desea
crear un índice jerárquico en el eje de concatenación. Para hacer esto, usar el argumento `keys`:

In [62]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [63]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


En caso de combinar series a lo largo `axis="columns"`, las `keys` se convertiran en los encabezados de columna:

In [64]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


La misa lógica se extiende a los objetos DataFrame:

In [65]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=list("abc"),
                  columns=["one", "two"])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=list("ac"),
                  columns=["three", "four"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [66]:
df2

,three,four
a,5,6
c,7,8


In [67]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Aquí el argumento `keys` se usa para crear un índice jerárquico donde el primer nivel se puede usar para identificar
cada uno de los objetos concatenados en el DataFrame.

Si pasa un diccionario de objetos en lugar de una lista, las claves del diccionario se utilizarán para la opción
`keys`:

In [68]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Hay argumentos adicionales que rigen cómo se crea el índice jerárquico. Por ejemplo, podemos nombrar los niveles de
eje creados con el argumento `name`:

In [69]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
         names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

Una última consideración se refiere a DataFrame en los que el índice de fila no contiene ningún dato relevante:

In [70]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                  columns=list("abcd"))

df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                  columns=list("bda"))

df1

,a,b,c,d
0,-0.094208,-0.071810,-0.111248,0.790240
1,-0.903595,-1.019421,0.771865,0.031050
2,-1.485661,-0.869526,-0.184135,0.762561


In [71]:
df2

,b,d,a
0,-0.804911,0.945494,-0.394523
1,1.049132,-1.441046,0.472894


En este caso, puede pasar `ingore_index=True`, que descarta los índices de cada DataFrame y concatena los datos solo
en las columnas, asignando un nuevo índice predeterminado:

In [72]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.094208,-0.071810,-0.111248,0.790240
1,-0.903595,-1.019421,0.771865,0.031050
2,-1.485661,-0.869526,-0.184135,0.762561
3,-0.394523,-0.804911,NaN,0.945494
4,0.472894,1.049132,NaN,-1.441046


### Combinando datos con superposición
Hay otras situaciones de combinación de datos que no se puede expresar como una operación de fusión o concatenación.
Puede tener dos conjuntos de datos con índices que se superponen en su totalidad o en parte. La función `where`
realiza el equivalente a orientado a matriz de una expresión if-else:

In [73]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
             index=list("fedcba"))

b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
             index=list("abcdef"))

a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [74]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [75]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [76]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

Con DataFrame, `combine_first` hace lo mismo columna por columna, por lo que puede pensarlo como "patching" datos
faltantes en el objeto que llama con datos del objeto que pasao:

In [77]:
df1 = pd.DataFrame({"a": [1, np.nan, 5., np.nan],
                   "b": [np.nan, 2., np.nan, 6.],
                   "c": range(2, 18, 4)})

df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                   "b": [np.nan, 3., 4., 6., 8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [78]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [79]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


La salida de `combine_first` con DataFrame tendrá la unión de todos los nombres de columna.

### Reformado y pivoteo
Hay una sere de operaciones básicas para reorganizar datos tabulares. Estos se conocen como _remodelar_ o
_pivote_.

### Reformado con indexación jarárquica
La indexación jerárquica proporciona una forma consistente de reorganizar los datos en un DataFrame. Hay dos
acciones principales:

`stack`: este rota o pivota desde las columnas en los datos de las filas.

`unstack`: esto pivota desde las filas hacia las columnas.

Considere un DataFrame pequeño con matrices de cadenas como índices de filas y columnas:

In [80]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                   index=pd.Index(["Ohio", "Colorado"], name="state"),
                   columns=pd.Index(["one", "two", "three"],
                                   name="number"))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Usando el método `stack` en estos datos gira las columnas en las filas, produciendo una serie:

In [81]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

Desde una serie indexada jerárquicamente, puede reorganizar los datos en un DataFrame con `unstack`:

In [82]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Puede desempaquetar un nivel diferente pasando un número de nivel o nombre:

In [83]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [84]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Desapilamiento podría introducir datos faltantes si todos los valores en el nivel se encuentran en cada subgrupo:

In [85]:
s1 = pd.Series([0, 1, 2, 3], index=list("abcd"), dtype="Int64")
s2 = pd.Series([4, 5, 6], index=list("cde"), dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

Apilar filtra los datos daltantes de forma predeterminanda, por lo que la operación es más fácilmente invertible:

In [86]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [87]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [88]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

Cuando se despega en un DataFrame, el nivel despegado se convierte en el nivel más bajo en el resultado:

In [89]:
df = pd.DataFrame({"left": result, "right": result + 5},
                 columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [90]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [91]:
df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivotar formato "long" a "wide"
Una forma común de almacenar múltiples series de tiempo en bases de datos y archivos CSV es lo que a veces se llama
formato _largo_ o _apilado_.

In [92]:
data = pd.read_csv("../files/examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


`pd.PeriodIndex` (que representa invervalos de tiempo en lugar de puntos de tiempo) para combinar `year` y `quarter`
para establecer el índice en el que consiste valores `datetime` al final de cada trimestre:

In [93]:
periods = pd.PeriodIndex(year=data.pop("year"),
                        quarter=data.pop("quarter"),
                        name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [94]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Se usa el método `pop` en el DataFrame, que devuelve una columna mientras elimina del DataFrame al mismo tiempo.

Luego, seleccionamos un subconjunto de columnas y se da el nombre de `"item"` a las columnas:

In [95]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Se remodela con `stack` para que convierta los nuevos niveles de índice en columnas con `reset_index`, y finalmente
dar a la columna que contiene valores de datos el nombre de `"value"`:

In [96]:
long_data = (data.stack().reset_index().rename(columns={0: "value"}))
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


En este llamado formato _largo_ para múltiples series de tiempo, cada fila de la tabla representa una sola
observación.

Los datos se almacenan con frecuencia de esta manera en bases de datos SQL relacionales, ya que es un esquema fijo
(nombres de columna y tipos de dato) permite el número de valores distintos en la columna `item` para cambiar a medida
que se agregan datos a la tabla. El ejemplo anterior, `date` e `item` generalmente serían las claves principales
(en el lenguaje de las bases de datos), que ofrece integridad relacional y uniones más fáciles. En algunos casos,
los datos pueden ser más difíciles de trabajar en este formato; es posible que prefiera tener un marco de datos que
contenga una columan por distinto valor `item` indexado por marcas de tiempo el la columna `date`. El método `pivot`
de DataFrame realiza exactamente esta transformación:

In [97]:
pivoted = long_data.pivot(index="date", columns="item",
                   values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Los dos primeros valores pasados son las columnas que se utilizarán, respectivamente, como el índice de la fila y
columna, y finalmente una columna de valor opcional para llenar el DataFrame. Supogamos que tenía dos columnas de
valor que quería remodelar simultáneamente:

In [98]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.881760
1,1959-01-01,infl,0.000,-0.763524
2,1959-01-01,unemp,5.800,0.586080
3,1959-04-01,realgdp,2778.801,1.834794
4,1959-04-01,infl,2.340,-0.529831
5,1959-04-01,unemp,5.100,-0.857186
6,1959-07-01,realgdp,2775.488,1.461932
7,1959-07-01,infl,2.740,-1.591663
8,1959-07-01,unemp,5.300,-0.764632
9,1959-10-01,realgdp,2785.204,1.899881


Al omitir el último elemento, se obtiene un DataFrame con columnas jerárquicas:

In [99]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.763524  0.881760  0.586080
1959-04-01  2.34  2778.801   5.1 -0.529831  1.834794 -0.857186
1959-07-01  2.74  2775.488   5.3 -1.591663  1.461932 -0.764632
1959-10-01  0.27  2785.204   5.6  0.361517  1.899881  1.058385
1960-01-01  2.31  2847.699   5.2 -0.164445  1.571683  0.015153

In [100]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Tenga en cuenta que `pivot` es equivalente a crear un índice jerárquico usando `set_index` seguido de una llamada a
`unstack`:

In [101]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")

unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.763524  0.881760  0.586080
1959-04-01  2.34  2778.801   5.1 -0.529831  1.834794 -0.857186
1959-07-01  2.74  2775.488   5.3 -1.591663  1.461932 -0.764632
1959-10-01  0.27  2785.204   5.6  0.361517  1.899881  1.058385
1960-01-01  2.31  2847.699   5.2 -0.164445  1.571683  0.015153

### Pivotar formato "wide" a "long"
Una operación inversa para `pivot` es `pd.melt`. En lugar de transformar una columna en muchas de un nuevo DataFrame,
fusiona varias columnas en una , produciendo un DataFrame que es más largo que la entrada.

In [102]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                  "A": range(1, 4),
                  "B": range(4, 7),
                  "C": range(7, 10)})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


La columna `key` puede ser un indicador de grupo, y las otras columnas son valores de datos. Cuando se usa `pd.melt`,
debemos indicar qué columna (si la hay) son indicadores de grupo. Usar `key` como único indicador de grupo:

In [103]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [104]:
# usar pivot para remodelar al diseño original
reshaped = melted.pivot(index="key", columns="variable", values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [105]:
# usar reset_index para mover de nuevo los datos a una columna
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


También se puede especificar un subconjunto de columnas para usar `value` como columnas:

In [106]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [107]:
# usar sin identificadores de grupo
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [108]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
